In [ ]:
%pip install structlog
%pip install opencensus-ext-azure
%pip install python-dotenv
%pip install -U structlog azure-monitor-opentelemetry python-dotenv


## AzureLogHandler


In [61]:
import os
from dotenv import load_dotenv
import structlog
from opencensus.ext.azure.log_exporter import AzureLogHandler
import logging

load_dotenv(".env")

logging.basicConfig(level=logging.DEBUG, force=True)

logger = structlog.get_logger()
connection_string = os.getenv("APPLICATIONINSIGHTS_CONNECTION_STRING")


In [62]:
# ロギング設定
logger = logging.getLogger("ApplicationInsights")
handler = AzureLogHandler(connection_string=connection_string)
logger.addHandler(handler)

# ログの送信例
logger.info("This is an INFO log sent to Azure Application Insights")
logger.warning("This is a WARNING log sent to Azure Application Insights")
logger.error("This is an ERROR log sent to Azure Application Insights")


INFO:ApplicationInsights:This is an INFO log sent to Azure Application Insights
ERROR:ApplicationInsights:This is an ERROR log sent to Azure Application Insights


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): japaneast-0.in.applicationinsights.azure.com:443
DEBUG:urllib3.connectionpool:https://japaneast-0.in.applicationinsights.azure.com:443 "POST //v2.1/track HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): japaneast-0.in.applicationinsights.azure.com:443
DEBUG:urllib3.connectionpool:https://japaneast-0.in.applicationinsights.azure.com:443 "POST //v2.1/track HTTP/1.1" 200 None


In [64]:
try:
    # 例外を発生させるコード
    raise UnicodeDecodeError("utf-8", b"", 1, 2, "test")
except Exception as e:
    logger.exception("UnicodeDecodeError", exc_info=True)


ERROR:ApplicationInsights:UnicodeDecodeError
Traceback (most recent call last):
  File "/var/folders/27/6hk2mfzs5vv9ysvns0ydhx2m0000gn/T/ipykernel_9744/52819273.py", line 3, in <module>
    raise UnicodeDecodeError("utf-8", b"", 1, 2, "test")
UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 1-1: test


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): japaneast-0.in.applicationinsights.azure.com:443
DEBUG:urllib3.connectionpool:https://japaneast-0.in.applicationinsights.azure.com:443 "POST //v2.1/track HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): japaneast-0.in.applicationinsights.azure.com:443
DEBUG:urllib3.connectionpool:https://japaneast-0.in.applicationinsights.azure.com:443 "POST //v2.1/track HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): japaneast-0.in.applicationinsights.azure.com:443
DEBUG:urllib3.connectionpool:https://japaneast-0.in.applicationinsights.azure.com:443 "POST //v2.1/track HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): japaneast-0.in.applicationinsights.azure.com:443
DEBUG:urllib3.connectionpool:https://japaneast-0.in.applicationinsights.azure.com:443 "POST //v2.1/track HTTP/1.1" 200 None


## structlog + AzureLogHandler


In [104]:
import os
from dotenv import load_dotenv
import structlog
from opencensus.ext.azure.log_exporter import AzureLogHandler
import logging
from opencensus.ext.azure.log_exporter import AzureLogHandler
from opencensus.ext.azure.log_exporter import AzureEventHandler

load_dotenv(".env")

logging.basicConfig(level=logging.DEBUG, force=True)

logger = structlog.get_logger()
connection_string = os.getenv("APPLICATIONINSIGHTS_CONNECTION_STRING")

# 既存のロギングハンドラーをクリア
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# ロガーの作成
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# コンソールハンドラーの作成
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Azure Log Handlerの作成
azure_handler = AzureLogHandler(connection_string=connection_string)
azure_handler.setLevel(logging.DEBUG)


# Telemetry Processorを定義してoperation_nameを設定
def set_operation_name(envelope):
    envelope.tags["ai.operation.name"] = "logger_test"


# Telemetry Processorをハンドラーに追加§
azure_handler.add_telemetry_processor(set_operation_name)

# structlogのProcessorFormatterを定義
# コンソールハンドラー用（色付き）
console_formatter = structlog.stdlib.ProcessorFormatter(
    processor=structlog.dev.ConsoleRenderer(colors=True),
    foreign_pre_chain=[
        structlog.processors.TimeStamper(fmt="iso"),
        structlog.stdlib.add_log_level,
        structlog.processors.ExceptionPrettyPrinter(),
    ],
)

# Azureハンドラー用（JSON形式）
azure_formatter = structlog.stdlib.ProcessorFormatter(
    processor=structlog.processors.JSONRenderer(),
    foreign_pre_chain=[
        structlog.processors.TimeStamper(fmt="iso"),
        structlog.stdlib.add_log_level,
        structlog.processors.StackInfoRenderer(),
        structlog.processors.format_exc_info,
    ],
)

# ハンドラーにフォーマッターを設定
console_handler.setFormatter(console_formatter)
azure_handler.setFormatter(azure_formatter)

# ロガーにハンドラーを追加
logger.addHandler(console_handler)
logger.addHandler(azure_handler)

# structlogの設定
structlog.configure(
    processors=[
        structlog.stdlib.add_logger_name,
        structlog.stdlib.add_log_level,
        structlog.processors.CallsiteParameterAdder(
            parameters=[
                structlog.processors.CallsiteParameter.FILENAME,  # Adds the filename
                structlog.processors.CallsiteParameter.LINENO,  # Adds the line number
            ]
        ),
        structlog.processors.TimeStamper(fmt="iso"),
        structlog.processors.StackInfoRenderer(),
        structlog.stdlib.ProcessorFormatter.wrap_for_formatter,
    ],
    logger_factory=structlog.stdlib.LoggerFactory(),
)

# structlogロガーの取§§
logger = structlog.get_logger()


In [105]:
# 使用例
logger.info("struct-test-info", data={"id": 1}, path="/path/to/resource")
logger.warning("struct-test-warning", dta={"id": 1}, path="/path/to/resource")
logger.debug("struct-test-debug", data={"id": 1}, path="/path/to/resource")
logger.error("struct-test-error", data={"id": 1}, path="/path/to/resource")


2024-11-19T07:43:14.667170Z [info     ] struct-test-info               [__main__] data={'id': 1} filename=3817302984.py lineno=2 path=/path/to/resource
2024-11-19T07:43:14.671665Z [warning  ] struct-test-warning            [__main__] dta={'id': 1} filename=3817302984.py lineno=3 path=/path/to/resource
2024-11-19T07:43:14.673115Z [error    ] struct-test-error              [__main__] data={'id': 1} filename=3817302984.py lineno=5 path=/path/to/resource


In [109]:
try:
    # 例外を発生させるコード
    raise UnicodeDecodeError("utf-8", b"", 1, 2, "test")
except Exception as e:
    logger.exception("StructLogUnicodeDecodeError", exc_info=True)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 try:                                                                                         │
│   2 │   # 例外を発生させるコード                                                                 │
│ ❱ 3 │   raise UnicodeDecodeError("utf-8", b"", 1, 2, "test")                                     │
│   4 except Exception as e:                                                                       │
│   5 │   logger.exception("StructLogUnicodeDecodeError", exc_info=True)                           │
│   6                                                                                              │
│                                                                                                  │
│ ╭─────────────────────────────────────────── locals ───────────────────────────────────────────╮ │
│ │                       _ = ''                                                                 │ │
│ │                     _dh = [PosixPath('/Users/takeshiiijima/github/python_logger')]           │ │
│ │                      _i = 'try:\n    # 例外を発生させるコード\n    raise                     │ │
│ │                           UnicodeDecodeError("utf-8", b"", 1, 2, "test")\n'+74               │ │
│ │                     _i1 = 'import os\nfrom dotenv import load_dotenv\nimport structlog\nfrom │ │
│ │                           opencensus.ext.az'+233                                             │ │
│ │                    _i10 = 'try:\n    # 例外を発生させるコード\n    raise                     │ │
│ │                           UnicodeDecodeError("utf-8", b"", 1, 2, "test")\n'+65               │ │
│ │                   _i100 = '# 使用例\nlogger.info("struct-test-info", data={"id": 1},         │ │
│ │                           path="/path/to/resource")\n'+230                                   │ │
│ │                   _i101 = 'try:\n    # 例外を発生させるコード\n    raise                     │ │
│ │                           UnicodeDecodeError("utf-8", b"", 1, 2, "test")\n'+89               │ │
│ │                   _i102 = '# 使用例\nlogger.info("struct-test-info", data={"id": 1},         │ │
│ │                           path="/path/to/resource")\n'+230                                   │ │
│ │                   _i103 = 'try:\n    # 例外を発生させるコード\n    raise                     │ │
│ │                           UnicodeDecodeError("utf-8", b"", 1, 2, "test")\n'+89               │ │
│ │                   _i104 = 'import os\nfrom dotenv import load_dotenv\nimport structlog\nfrom │ │
│ │                           opencensus.ext.az'+2700                                            │ │
│ │                   _i105 = '# 使用例\nlogger.info("struct-test-info", data={"id": 1},         │ │
│ │                           path="/path/to/resource")\n'+230                                   │ │
│ │                   _i106 = 'try:\n    # 例外を発生させるコード\n    raise                     │ │
│ │                           UnicodeDecodeError("utf-8", b"", 1, 2, "test")\n'+89               │ │
│ │                   _i107 = 'try:\n    # 例外を発生させるコード\n    raise                     │ │
│ │                           UnicodeDecodeError("utf-8", b"", 1, 2, "test")\n'+89               │ │
│ │                   _i108 = 'try:\n    # 例外を発生させるコード\n    raise                     │ │
│ │                           UnicodeDecodeError("utf-8", b"", 1, 2, "test")\n'+74               │ │
│ │                   _i109 = 'try:\n    # 例外を発生させるコード\n    raise                     │ │
│ │                           UnicodeDecodeError("utf-8", b"", 1, 2, "test")\n'+89               │ │
│ │                    _i11 = 'import os\nfrom dotenv import load_dotenv\nimport structlog\nfrom │ │
│ │                           opencensus.ext.az'+2175               

2024-11-19T07:53:46.634296Z [error    ] StructLogUnicodeDecodeError    [__main__] filename=561646677.py lineno=5



In [25]:
# try:
#     1 / 0  # エラーを発生！
# except ZeroDivisionError:
#     struct_logger.exception(
#         "structlog-structlog-structlog-structlog-error", exc_info=True
#     )
